<a href="https://colab.research.google.com/github/100jy/voice_competition/blob/master/baseline_%EC%BD%94%EB%93%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

구글 드라이브 연동

In [ ]:
#구글 드라이브 연동
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


모듈 임포트

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy
import tensorflow as tf
import keras
from tqdm import tqdm
from glob import glob
from scipy.io import wavfile
from keras.models import Sequential
from keras.layers import Dense, Conv1D, MaxPooling1D, Flatten, BatchNormalization
from keras.callbacks import ModelCheckpoint, EarlyStopping
import librosa

Using TensorFlow backend.


# 음성에서 정형 데이터화

1. 음성신호 상태로 이용하는 방법 
2. MFCC이용하는 방법
3. spectogram 이용하는 방법

In [ ]:
# 데이터 로드 함수 정의 
def make_data(datalist, feature = 'signal'):

    arr = []
    #signal data
    if feature == 'signal':
        for file in tqdm(datalist):
            fs, data = wavfile.read(file)
            arr.append(data)
  
    #spectogam data
    elif feature == 'spectogram':
        for file in tqdm(datalist):
            x, sr = librosa.load(file)
            spec = librosa.stft(x)
            arr.append(spec)
  
    #MFCC data
    elif feature == 'MFCC':
        for file in tqdm(datalist):
            x, sr = librosa.load(file)
            mfcc = librosa.feature.mfcc(x, sr=sr, n_mfcc=40, fmax=3000)
            arr.append(mfcc)
  
    else:
        raise ValueError

    result = np.array(arr)
    
    return result 

VAD하면 성능이 더 좋지 않을까???


트레인 셋 저장

In [ ]:
#signal 형태로 사용
raw_data = glob('data/train/*.wav')
#signal
signal = make_data(raw_data)
pd.DataFrame(signal).to_pickle('data/x_trian.pickle')
#specto
specto = make_data(raw_data, feature =  'spectogram')
pd.DataFrame(specto).to_pickle('data/x_trian_spectogram.pickle')
#MFCC
MFCC = make_data(raw_data, feature =  'MFCC')
pd.DataFrame(MFCC).to_pickle('data/x_trian_MFCC.pickle')
#feature = 
#x_train = pd.read_csv('data/train/x_train_{}.pickle'.format(feature))

테스트 셋 저장

In [ ]:
#signal 형태로 사용
raw_data = glob('drive/My Drive/data/test/*.wav')
#signal
signal = make_data(raw_data)
pd.DataFrame(signal).to_pickle('drive/My Drive/data/x_test.pickle')

데이터 불러오기


In [ ]:
#원본
x_train = pd.read_pickle('drive/My Drive/data/x_trian.pickle')
x_train = x_train.values
#열수가 1/8 토막(메모리가 부족함...) 
x_train = x_train[:,::8]

In [ ]:
#MFCC
x_train = pd.read_pickle('drive/My Drive/data/x_trian_MFCC.pickle')
x_train = x_train.values

In [ ]:
# (0,1)로 정규화
x_train = list((map(lambda x : x / max(x),x_train)))
x_train =  np.array(x_train)
# 모델 넣기위한 reshape
## 채널 늘려보는 것도 의미있을듯...
x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], 1)

In [ ]:
y_train = pd.read_csv('drive/My Drive/data/train_answer.csv',index_col=0)
y_train = y_train.values

x_train.shape, y_train.shape

((100000, 1760, 1), (100000, 30))

In [ ]:
#CNN
model = Sequential()
model.add(Conv1D(16,32,activation='relu', input_shape = (x_train.shape[1], x_train.shape[2])))
model.add(MaxPooling1D())
model.add(Conv1D(16,32,activation='relu'))
model.add(MaxPooling1D())
model.add(Conv1D(16,32,activation='relu'))
model.add(MaxPooling1D())
model.add(Conv1D(16,32,activation='relu'))
model.add(MaxPooling1D())
model.add(Conv1D(16,32,activation='relu'))
model.add(Flatten())

#fully connected layer
model.add(Dense(256, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(256, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(256, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(30, activation='softmax'))
model.compile(loss=keras.losses.KLDivergence(), optimizer = 'adam')

In [ ]:
model_path = 'drive/My Drive/data/model/'
if not os.path.exists(model_path):
  os.mkdir(model_path)

# validattion 기준 모델 갱신
model_file = model_path + 'epoch_{epoch:03d}_val_{val_loss:3f}.hdf5'
checkpoint = ModelCheckpoint(filepath = model_file, monitor = 'val_loss', verbose = 1, save_best_only =True)

#10회간 validatation 좋아지지 않으면 early stop
early_stop = EarlyStopping(monitor = 'val_loss', patience = 10)

history = model.fit(x_train, y_train, epochs = 100, batch_size = 256, validation_split = 0.2, shuffle = True,
                    callbacks = [checkpoint, early_stop])

Train on 80000 samples, validate on 20000 samples
Epoch 1/100
80000/80000 [==============================] - 368s 5ms/step - loss: 1.9581 - val_loss: 1.9758

Epoch 00001: val_loss improved from inf to 1.97577, saving model to drive/My Drive/data/model/epoch_001_val_1.975766.hdf5
Epoch 2/100
80000/80000 [==============================] - 363s 5ms/step - loss: 1.8115 - val_loss: 1.9083

Epoch 00002: val_loss improved from 1.97577 to 1.90827, saving model to drive/My Drive/data/model/epoch_002_val_1.908270.hdf5
Epoch 3/100
80000/80000 [==============================] - 365s 5ms/step - loss: 1.7517 - val_loss: 2.2215

Epoch 00003: val_loss did not improve from 1.90827
Epoch 4/100
80000/80000 [==============================] - 364s 5ms/step - loss: 1.6978 - val_loss: 2.5512

Epoch 00004: val_loss did not improve from 1.90827
Epoch 5/100
80000/80000 [==============================] - 365s 5ms/step - loss: 1.6601 - val_loss: 2.1594

Epoch 00005: val_loss did not improve from 1.90827
Epoch 6/1

In [ ]:
# 훈련 결과를 확인합니다.
plt.plot(history.epoch, history.history['loss'], '-o', label='training_loss')
plt.plot(history.epoch, history.history['val_loss'], '-o', label='validation_loss')
plt.legend()
plt.xlim(left=0)
plt.xlabel('epochs')
plt.ylabel('loss')
plt.show()

전처리 필요성, 모델 CNN 이용
